In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import scipy
from scipy import interpolate
import pandas as pd
import itertools
from root_numpy import root2array, root2rec, tree2rec, array2root


In [ ]:
def get_data_frame(tfile,ttree):
    rec = root2array(tfile,ttree)
    return pd.DataFrame(rec)

print 'Reading in real data...'

michel_cluster_loc = "/Users/vgenty/git/LARLITE/UserDev/MichelReco/MichelCluster/mac/michel_clusters.root"
REALDATA = get_data_frame(michel_cluster_loc,"out_tree")


In [ ]:
handscan_textfilename = '/Users/vgenty/git/LARLITE/UserDev/MichelReco/MichelCluster/mac/michelcluster_cchit_fromfuzzycluster_handscanning_results.txt'
hscan_results = pd.read_csv(handscan_textfilename,
                            delimiter=' ',
                            header=None,
                            names=['_run',
                                   '_subrun',
                                   '_event',
                                   '_clus_idx',
                                   'good_or_bad'])

In [ ]:
REALDATA2 = REALDATA.merge(hscan_results,how='outer')

REALDATA2.fillna(-1,inplace=True)

In [ ]:
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
print "\t!!!!~~~~~~~~~~~~~~NO CUTS~~~~~~~~~~~~~!!!!"
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
#kaleko: this used to be _largest_cluster_charge... cosmic_ana doesn't make that variable
S = REALDATA2.query    ('_michel_clustered_charge > 0 and good_or_bad == 1')
B = REALDATA2.query    ('_michel_clustered_charge > 0 and good_or_bad == 0')

plt.figure(figsize=(16,24))

plt.subplot(3,2,1)
plt.grid(True)
plt.hist(np.array(B['_lowest_chi']),bins=100,range=(0,1),normed=1,color='r',alpha=0.5,label="Handscan Background")
plt.hist(np.array(S['_lowest_chi']),bins=100,range=(0,1),normed=1,color='b',alpha=0.5,label="Handscan Signal")
plt.tick_params(labelsize=20)
plt.xlabel('Lowest Chi',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

plt.subplot(3,2,2)
plt.grid(True)
plt.hist(np.array(B['_mean_chi']),bins=100,range=(0,1),normed=1,color='r',alpha=0.5,label="Handscan Background")
plt.hist(np.array(S['_mean_chi']),bins=100,range=(0,1),normed=1,color='b',alpha=0.5,label="Handscan Signal")
plt.tick_params(labelsize=20)
plt.xlabel('Mean Chi',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

plt.subplot(3,2,3)
plt.grid(True)
plt.hist(np.array(B['_chi_at_boundary']),bins=100,range=(0,1),normed=1,color='r',alpha=0.5,label="Handscan Background")
plt.hist(np.array(S['_chi_at_boundary']),bins=100,range=(0,1),normed=1,color='b',alpha=0.5,label="Handscan Signal")
plt.tick_params(labelsize=20)
plt.xlabel('Chi at Boundary',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

plt.subplot(3,2,4)
plt.grid(True)
#kaleko: _n_hits_in_largest_cluster_michel => _michel_n_hits
plt.hist(np.array(B['_michel_n_hits']),bins=100,range=(0,100),normed=1,color='r',alpha=0.5,label="Handscan Background")
plt.hist(np.array(S['_michel_n_hits']),bins=100,range=(0,100),normed=1,color='b',alpha=0.5,label="Handscan Signal")
plt.tick_params(labelsize=20)
plt.xlabel('N Hits in Michel',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.legend(fontsize=20)

#kaleko: this used to be _largest_cluster_charge... cosmic_ana doesn't make that variable
plt.subplot(3,2,5)
plt.grid(True)
plt.hist(np.array(B['_michel_clustered_charge'])*0.008*1.4,bins=100,range=(0,200),normed=1,color='r',alpha=0.5,label="Handscan Background")
plt.hist(np.array(S['_michel_clustered_charge'])*0.008*1.4,bins=100,range=(0,200),normed=1,color='b',alpha=0.5,label="Handscan Signal")
plt.tick_params(labelsize=20)
plt.xlabel('Energy [MeV]',fontsize=20)
plt.ylabel('Count/2 MeV',fontsize=20)
plt.legend(fontsize=20)
print "entries = %d, underflow = %d, overflow = %d"%(len(S['_michel_clustered_charge']),S.query('_michel_clustered_charge*0.008*1.4<0').shape[0],S.query('_michel_clustered_charge*0.008*1.4>200').shape[0])
plt.show()

In [ ]:
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
print "\t!!!!~~~~~~~~~~~~CORRELATIONS~~~~~~~~~~!!!!"
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"

plt.figure(figsize=(16,32))

plt.subplot(5,2,1)
Q1 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_lowest_chi']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Signal",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('Lowest Chi',fontsize=20)

plt.subplot(5,2,2)
Q1 = plt.hist2d(np.array(B['_michel_clustered_charge'])*0.008*1.4,
                np.array(B['_lowest_chi']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Background",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('Lowest Chi',fontsize=20)

plt.subplot(5,2,3)
Q3 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_chi_at_boundary']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Signal",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('Chi at Boundary',fontsize=20)

plt.subplot(5,2,4)
Q3 = plt.hist2d(np.array(B['_michel_clustered_charge'])*0.008*1.4,
                np.array(B['_chi_at_boundary']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Background",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('Chi at Boundary',fontsize=20)

plt.subplot(5,2,5)
Q3 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_mean_chi']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Signal",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('Mean Chi',fontsize=20)

plt.subplot(5,2,6)
Q3 = plt.hist2d(np.array(B['_michel_clustered_charge'])*0.008*1.4,
                np.array(B['_mean_chi']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Background",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('Mean Chi',fontsize=20)

plt.subplot(5,2,7)
Q3 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_rms_chi']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Signal",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('RMS Chi',fontsize=20)

plt.subplot(5,2,8)
Q3 = plt.hist2d(np.array(B['_michel_clustered_charge'])*0.008*1.4,
                np.array(B['_rms_chi']), 
                bins = 100,
                range=np.array([(0, 100), (0,1)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Background",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('RMS Chi',fontsize=20)

plt.subplot(5,2,9)
Q3 = plt.hist2d(np.array(S['_michel_clustered_charge'])*0.008*1.4,
                np.array(S['_michel_n_hits']), 
                bins = 100,
                range=np.array([(0, 100), (0,100)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Signal",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('N Hits in Michel Cluster',fontsize=20)

plt.subplot(5,2,10)
Q3 = plt.hist2d(np.array(B['_michel_clustered_charge'])*0.008*1.4,
                np.array(B['_michel_n_hits']), 
                bins = 100,
                range=np.array([(0, 100), (0,100)]))
plt.colorbar()
plt.tick_params(labelsize=20)
plt.title("Handscanned Background",fontsize=20)
plt.xlabel('Energy',fontsize=20)
plt.ylabel('N Hits in Michel Cluster',fontsize=20)

plt.show()


In [ ]:
#THIS CELL IS OUTDATED AND DOES NOT APPLY TO REAL DATA

print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
print "\t!!!!~~~~~~~~~~~~WITH CUTS~~~~~~~~~~~~~!!!!"
print "\t!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"

print 'Applying precuts to signal...'
print 'Queried charge > 0. Events left=',S.shape[0]
S = S.query('_michel_n_hits >= 10')
print 'Queried n_hits >= 10. Events left=',S.shape[0]
S = S.query('_lowest_chi < 0.28')
print 'Queried lowest chi < 0.28. Events left=',S.shape[0]
S = S.query('_chi_at_boundary < 0.68')
print 'Queried chi at boundary < 0.68. Events left=',S.shape[0]
S = S.query('_michel_clustered_charge < 8929')
print 'Queried charge < 8929. Events left=',S.shape[0]
print ''
print 'Applying precuts to background...'
print 'Queried charge > 0. Events left=',B.shape[0]
B = B.query('_michel_n_hits >= 10')
print 'Queried n_hits >= 10. Events left=',B.shape[0]
B = B.query('_lowest_chi < 0.28')
print 'Queried lowest chi < 0.28. Events left=',B.shape[0]
B = B.query('_chi_at_boundary < 0.68')
print 'Queried chi at boundary < 0.68. Events left=',B.shape[0]
B = B.query('_michel_clustered_charge < 8929')
print 'Queried charge < 8929. Events left=',B.shape[0]

plt.figure(figsize=(12,6))
plt.hist(np.array(B['_michel_clustered_charge'])*0.008*1.4,bins=20,range=(0,200),color='r',alpha=0.5,label="Background")
plt.hist(np.array(S['_michel_clustered_charge'])*0.008*1.4,bins=20,range=(0,200),color='b',alpha=0.5,label="Signal")
plt.tick_params(labelsize=20)

plt.xlabel('Energy [MeV]',fontsize=20)
plt.ylabel('Count/2 MeV',fontsize=20)
plt.legend(fontsize=20)
plt.savefig('sample.pdf')
plt.show()


In [ ]:
df_event

In [ ]:
k = 4
df_event = S.iloc[[k]]

print "Estimated energy..."
print df_event['_michel_clustered_charge'].values*0.008*1.4
print "N Hits Michel..."
print df_event['_michel_n_hits'].values[0]


plt.figure(figsize=(24,24))

plt.subplot(3,2,1)

plt.plot(df_event["_Z"].values[0],
         df_event["_X"].values[0],
         "-",color="r")
plt.xlabel('Z',fontsize=20)
plt.ylabel('X',fontsize=20)
plt.tick_params(labelsize=20)

plt.subplot(3,2,2)
plt.plot(np.array(df_event["_Z"])[0],
         np.array(df_event["_X"])[0],
         "ro",color="r")
plt.plot(np.array(df_event["_michel_Z"])[0],
         np.array(df_event["_michel_X"])[0],
         "ro",color="b")

plt.xlabel('Z',fontsize=20)
plt.ylabel('X',fontsize=20)
plt.tick_params(labelsize=20)

plt.subplot(3,2,3)
plt.plot(np.array(df_event["_s_v"])[0],
         np.array(df_event["_q_v"])[0],
         "ro",color="r")
plt.xlabel('s' ,fontsize=20)
plt.ylabel('Q',fontsize=20)
plt.tick_params(labelsize=20)


plt.subplot(3,2,4)
plt.plot(np.array(df_event["_s_v"])[0],
         np.array(df_event["_t_q_v"])[0],
         "ro",color="r")
plt.xlabel('s' ,fontsize=20)
plt.ylabel('TQ',fontsize=20)
plt.tick_params(labelsize=20)

plt.subplot(3,2,5)
plt.plot(np.array(df_event["_s_v"])[0],
         np.array(df_event["_t_dqds_v"])[0],
         "ro",color="r")
plt.xlabel('s' ,fontsize=20)
plt.ylabel('TDqDs',fontsize=20)
plt.tick_params(labelsize=20)

plt.subplot(3,2,6)
plt.plot(np.array(df_event["_s_v"])[0],
         np.array(df_event["_chi_v"])[0],
         "ro",color="r")
plt.xlabel('s' ,fontsize=20)
plt.ylabel('chi2',fontsize=20)
plt.tick_params(labelsize=20)


plt.show()

